In [3]:
from scipy.spatial.transform import Rotation as R
import numpy as np

In [7]:
transformed_centers_path = r'G:\VS2022Projects\arap-volume-tracking-main\data\scene-max-4000/combined_mesh_0res_4000_004.xyz'
loaded_transformed_centers = np.loadtxt(transformed_centers_path)
with open(transformed_centers_path, 'r') as file:
    lines = file.readlines()
    print("".join(lines[:5])) 

-0.65743625 -2.2764578 0.16107747
-0.66250813 -2.2309651 1.5619742
-0.3686443 -0.302735 0.6004992
0.20994216 -0.18073219 0.2659464
-0.66087186 -2.758741 0.4889144



In [5]:
def get_dual_quaternions(original_centers, transformed_centers):
    moved_indices = []
    for i in range(len(original_centers)):
        if not (np.array_equal(original_centers[i], transformed_centers[i])):
            moved_indices.append(i)
    dual_quaternions = np.zeros((len(original_centers), 8), dtype=np.float32)
    inverse_dual_quaternions = np.zeros((len(original_centers), 8), dtype=np.float32)
    for i in moved_indices:
        original = original_centers[i]
        transformed = loaded_transformed_centers[i]
        
        rotation_quaternion = R.from_quat([0, 0, 0, 1])  
    
        translation = transformed - original
        
        rotation_quat = rotation_quaternion.as_quat()
        
        translation_quat = np.hstack((translation, [0]))
        dual_quat = np.hstack((rotation_quat, 0.5 * translation_quat))
        dual_quaternions[i] = dual_quat
        
        rotation_conjugate = np.hstack((-rotation_quat[:3], rotation_quat[3]))
        inv_translation_quat = np.hstack((translation, [0]))
        inverse_dual_quat = np.hstack((rotation_conjugate, -0.5 * inv_translation_quat))
        inverse_dual_quaternions[i] = inverse_dual_quat
        
    return moved_indices, dual_quaternions, inverse_dual_quaternions

In [9]:
transformed_centers_path = r'G:\VS2022Projects\arap-volume-tracking-main\data\scene-max-4000/combined_mesh_0res_4000_003.xyz'
loaded_transformed_centers = np.loadtxt(transformed_centers_path)

for i in range (1, 8):
    centers_path = fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking/centers/combined_mesh_0res_4000_{i:03}.xyz'
    loaded_centers = np.loadtxt(centers_path)
    print(centers_path)
    
    indices, dual_quaternions, inverse_dual_quaternions = get_dual_quaternions(loaded_centers, loaded_transformed_centers)
    
    indices_path = fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking/indices_{i:03}.txt'
    np.savetxt(indices_path, indices, fmt='%d')
    
    dual_quaternions_path = fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking\transformations_{i:03}.txt'
    with open(dual_quaternions_path, 'w') as file:
        for dq in dual_quaternions:
            dq_str = f"{dq[0]};{dq[1]};{dq[2]};{dq[3]};{dq[4]};{dq[5]};{dq[6]};{dq[7]}"
            file.write(dq_str + '\n')
            
    inverse_dual_quaternions_path = fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking\inverse_transformations_{i:03}.txt'
    with open(inverse_dual_quaternions_path, 'w') as file:
        for inverse_dq in inverse_dual_quaternions:
            inverse_dq_str = f"{inverse_dq[0]};{inverse_dq[1]};{inverse_dq[2]};{inverse_dq[3]};{inverse_dq[4]};{inverse_dq[5]};{inverse_dq[6]};{inverse_dq[7]}"
            file.write(inverse_dq_str + '\n')

G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking/centers/combined_mesh_0res_4000_001.xyz
G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking/centers/combined_mesh_0res_4000_002.xyz
G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking/centers/combined_mesh_0res_4000_003.xyz
G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking/centers/combined_mesh_0res_4000_004.xyz
G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking/centers/combined_mesh_0res_4000_005.xyz
G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking/centers/combined_mesh_0res_4000_006.xyz
G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking/centers/combined_mesh_0res_4000_007.xyz
